In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [8]:
#config_file = "configs/vgg_ssd300_voc0712_tdt4265_server.yaml"
#config_file = "configs/mnist_tdt4265_server_experimental.yaml"
# config_file = "configs/train_rdd2020_server.yaml"
config_file = "configs/train_rdd2020_server_experimental.yaml"
cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-19 12:10:09,395 SSD INFO: Loaded configuration file configs/train_rdd2020_server_experimental.yaml
2021-04-19 12:10:09,395 SSD INFO: Loaded configuration file configs/train_rdd2020_server_experimental.yaml
2021-04-19 12:10:09,398 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    THRESHOLD: 0.5
    BACKBONE:
        NAME: 'basic'
        PRETRAINED: False
        OUT_CHANNELS: [256, 512, 256, 256, 128, 128]
        INPUT_CHANNELS: 3
    PRIORS:
        MIN_SIZES: [[15, 15], [45, 45], [111, 111], [162, 162], [213, 213], [264, 264]]
        MAX_SIZES: [[45, 45], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]
        FEATURE_MAPS: [[63, 38], [32, 19], [16, 10], [8, 5], [4, 3], [2, 1]]
        STRIDES: [[8, 8], [16, 16], [32, 32], [64, 64], [100, 100], [300, 300]]

INPUT:
    IMAGE_SIZE: [500, 300]
DATASETS:
    TRAIN: ("rdd2020_train",)
    TEST: ("rdd2020_val", )
SOLVER:
    MAX_ITER: 120000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 1e-3
OUTPUT_DIR: 'outputs/rdd2020_experiment

In [ ]:
!pwd 
model = start_train(cfg)

/notebooks/TDT4265-Project/assignment4/SSD
Detector initialized. Total Number of params:  8.73M
Backbone number of parameters: 8.06M
SSD Head number of parameters: 663.8K
2021-04-19 12:10:14,993 SSD.trainer INFO: No checkpoint found.
2021-04-19 12:10:14,993 SSD.trainer INFO: No checkpoint found.
Dataset loaded. Subset: train, number of images: 8536
2021-04-19 12:10:15,113 SSD.trainer INFO: Start training ...
2021-04-19 12:10:15,113 SSD.trainer INFO: Start training ...
2021-04-19 12:10:17,387 SSD.trainer INFO: iter: 000010, lr: 0.00010, total_loss: 21.466 (21.466), reg_loss: 3.815 (3.815), cls_loss: 17.651 (17.651), time: 0.227 (0.227), eta: 7:33:19, mem: 10520M
2021-04-19 12:10:17,387 SSD.trainer INFO: iter: 000010, lr: 0.00010, total_loss: 21.466 (21.466), reg_loss: 3.815 (3.815), cls_loss: 17.651 (17.651), time: 0.227 (0.227), eta: 7:33:19, mem: 10520M
2021-04-19 12:10:19,039 SSD.trainer INFO: iter: 000020, lr: 0.00010, total_loss: 14.602 (18.034), reg_loss: 3.408 (3.612), cls_loss: 

100%|██████████| 366/366 [25:36<00:00,  4.20s/it]


2021-04-19 12:37:18,355 SSD.inference INFO: mAP: 0.0126
D00             : 0.0001
D10             : 0.0000
D20             : 0.0502
D40             : 0.0000

2021-04-19 12:37:18,355 SSD.inference INFO: mAP: 0.0126
D00             : 0.0001
D10             : 0.0000
D20             : 0.0502
D40             : 0.0000

2021-04-19 12:37:20,009 SSD.trainer INFO: iter: 000510, lr: 0.00010, total_loss: 10.480 (11.801), reg_loss: 2.804 (2.928), cls_loss: 7.677 (8.873), time: 154.089 (3.186), eta: 4 days, 9:45:01, mem: 10520M
2021-04-19 12:37:20,009 SSD.trainer INFO: iter: 000510, lr: 0.00010, total_loss: 10.480 (11.801), reg_loss: 2.804 (2.928), cls_loss: 7.677 (8.873), time: 154.089 (3.186), eta: 4 days, 9:45:01, mem: 10520M
2021-04-19 12:37:21,682 SSD.trainer INFO: iter: 000520, lr: 0.00010, total_loss: 10.614 (11.779), reg_loss: 2.794 (2.926), cls_loss: 7.821 (8.853), time: 0.167 (3.128), eta: 4 days, 7:48:53, mem: 10520M
2021-04-19 12:37:21,682 SSD.trainer INFO: iter: 000520, lr: 0.00010, tota

 30%|██▉       | 109/366 [07:31<18:19,  4.28s/it]

In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-04-15 10:20:59,235 SSD INFO: Start evaluating...
2021-04-15 10:21:01,146 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):


100%|██████████| 100/100 [00:04<00:00, 24.52it/s]


2021-04-15 10:21:06,107 SSD.inference INFO: mAP: 0.8952
0               : 0.9038
1               : 0.8619
2               : 0.8991
3               : 0.9034
4               : 0.9012
5               : 0.9012
6               : 0.9021
7               : 0.8900
8               : 0.9006
9               : 0.8886



[{'metrics': {'mAP': 0.8952024131918288,
   '0': 0.9038319683421214,
   '1': 0.8619123396332643,
   '2': 0.8991184457164805,
   '3': 0.9034108127494757,
   '4': 0.9012078430736787,
   '5': 0.9011639146270728,
   '6': 0.9021376905405668,
   '7': 0.8900135604682246,
   '8': 0.9006312166832495,
   '9': 0.8885963400841522}}]